# Cargando librería Pandas y documento cvs con datos

In [3]:
import pandas as pd

df= pd.read_csv("https://raw.githubusercontent.com/jdvelasq/datalabs/master/datasets/snsdata.csv",
               sep=",",
               thousands=None,
               decimal=".",
               encoding="latin-1",
              )

df.head(10)

,gradyear,gender,age,friends,basketball,football,soccer,softball,volleyball,swimming,...,blonde,mall,shopping,clothes,hollister,abercrombie,die,death,drunk,drugs
0,2006,M,18.982,7,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2006,F,18.801,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,2006,M,18.335,69,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,2006,F,18.875,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2006,NaN,18.995,10,0,0,0,0,0,0,...,0,0,2,0,0,0,0,0,1,1
5,2006,F,NaN,142,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
6,2006,F,18.930,72,0,0,0,0,0,0,...,0,2,0,0,2,0,0,0,0,0
7,2006,M,18.322,17,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
8,2006,F,19.055,52,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,2006,F,18.708,39,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


# Análisis exploratorio de los datos

In [5]:
# número de registros total
len(df)

30000

In [7]:
# Número de valores diferentes del tipo de género, F para femenino y M para masculino.
df.gender.value_counts()

F    22054
M     5222
Name: gender, dtype: int64

In [9]:
# Cantidad de registros que no tienen género, función isna.
df.gender.isna().sum()

2724

In [11]:
# El archivo contiene edades fuera de la población de interés del problema, 
# observamos estos datos con los valores estadísticos básicos en la columna edad.
# Se observa que existen registros que no tienen un valor en edad, que la edad mínima es 3.08 
# y la máxima 106.92
df.age.describe()

count    24914.000000
mean        17.993950
std          7.858054
min          3.086000
25%         16.312000
50%         17.287000
75%         18.259000
max        106.927000
Name: age, dtype: float64

In [12]:
# cantidad de registros nulos en columna edad, función isnull.
df.age.isnull().sum()

5086

In [14]:
# importamos la librería numpy
import numpy as np

# Selecionamos las personas entre 13 y 20 años, se descartan las demás edades.
# Función lambda, si x>13 borra los datos (aplica un nan), en caso contrario los mantiene, similar
# para los mayores o iguales a 20 años.
df['age']=df['age'].map(lambda x: np.nan if x<13 else x)
df['age']=df['age'].map(lambda x: np.nan if x>=20 else x)

# Verificamos que nuestros nuevos datos se encuentren en el intervalo de interés.
df.age.describe()

count    24477.000000
mean        17.252429
std          1.157465
min         13.027000
25%         16.304000
50%         17.265000
75%         18.220000
max         19.995000
Name: age, dtype: float64

In [16]:
# Utilizando nuevamente la función lambda cambiamos el valor de las variables del tipo género
# por 1 para femenino y 0 para masculino, de igual manera se crea la variable no_gender con valor 1
# para aquellos valores nulos en género y 0 de forma contraria.
df['female']=df['gender'].map(lambda x:1 if x=='F' else 0)
df['no_gender']=df['gender'].map(lambda x:1 if pd.isnull(x) else 0)

# Obtenemos la cantidad de  hombres y mujeres en la muestra.
df.gender.value_counts()

F    22054
M     5222
Name: gender, dtype: int64

In [18]:
# Valores que no tienen género.
df.gender.isna().sum()

2724

In [19]:
# En nuestros nuevos datos observamos la cantidad de mujeres (valor 1) y 
# la cantidad de hombres (valor 0), con lo cual obtenemos los 30,000 datos totales.
df.female.value_counts()

1    22054
0     7946
Name: female, dtype: int64

In [20]:
# De igual forma para la variables no_gender
df.no_gender.value_counts()

0    27276
1     2724
Name: no_gender, dtype: int64

In [21]:
# Promedio de edad sin contar los datos faltantes.
df.age.mean()

17.25242893328433

In [26]:
# Se puede construir una tabla con el campo año de graduación (gradyear) y
# en la otra columna la media de laedad, utilizamos groupby. Se observa que entre menor
# el año de graduación, mayor la edad.
df.groupby('gradyear')['age'].mean()

gradyear
2006    18.655858
2007    17.706172
2008    16.767701
2009    15.819573
Name: age, dtype: float64

In [28]:
# Completamos los valores de la edad utilizando 
# como variable el promedio de acuerdo con el año
# de graduación
# Creamos la variable ave_age en la cual se almacena la media por año de la
# edad por cada año de graduación.
ave_age=df.groupby("gradyear")['age'].mean()
# En la columna edad reemplazamos los datos nulos por la variable ave_age, en caso
# contrario el dato queda sin cambios.
df.age=[x if not pd.isnull(x) else ave_age[y] for x, y in zip(df.age, df.gradyear)]
# Observamos los nuevos descriptivos tras nuestros cambios.
df.age.describe()

count    30000.000000
mean        17.237326
std          1.141821
min         13.027000
25%         16.282000
50%         17.238000
75%         18.212000
max         19.995000
Name: age, dtype: float64

# Entrenamiento del modelo.

In [30]:
# Eliminamos las primeras cuatro columnas y únicamente se conservan
# aquellas que pueden representar intereses.
interests=df.iloc[:,4:].copy()
interests.head()

,basketball,football,soccer,softball,volleyball,swimming,cheerleading,baseball,tennis,sports,...,shopping,clothes,hollister,abercrombie,die,death,drunk,drugs,female,no_gender
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,0,0,0,0,0,0,0,0,0,0,...,2,0,0,0,0,0,1,1,0,1


In [31]:
# Realizamos un escalamiento de la matriz para tenerla en un rango (0,1).
# Importamos MinMaxScaler
from sklearn.preprocessing import MinMaxScaler
# los interéses escalados los guardamos en interests_z
scaler=MinMaxScaler()
interests_z=scaler.fit_transform(interests)
interests_z

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.06666667, 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.06666667, 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.06666667, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.08333333, 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ]])

In [32]:
# Utilizaremos el algortimo K-Means suponiendo 5 grupos, sobre la matriz escalada.
from sklearn.cluster import KMeans
df_clusters=KMeans(n_clusters=5, random_state=1).fit(interests_z)

# Evaluación del modelo

In [33]:
 # Contamos el número de individuos que se asignaron a cada clúster.
(pd.DataFrame(df_clusters.predict(interests_z)))[0].value_counts()

3    16472
0     5222
1     3095
2     2724
4     2487
Name: 0, dtype: int64

In [34]:
# Centros de los clústers
df_clusters.cluster_centers_

array([[ 1.31415167e-02,  2.98991446e-02,  7.78047293e-03,
         5.40698853e-04,  2.47578924e-03,  2.57595038e-03,
         9.57487553e-04,  1.69714669e-02,  6.88114388e-03,
         1.66443253e-02,  3.95761522e-03,  1.40767468e-03,
         4.57466275e-03,  5.18958254e-03,  2.26850897e-03,
         6.85561088e-03,  5.32130961e-03,  4.45666934e-03,
         1.08196093e-02,  1.20461053e-02,  5.41283470e-03,
         4.44796490e-03,  4.16826248e-03,  2.48946764e-03,
         4.88059871e-03,  3.42567769e-03,  7.43736509e-05,
         1.13302694e-02,  4.43926047e-03,  9.02432018e-03,
         3.63845270e-03,  2.84852547e-03,  8.10382647e-03,
         7.44104612e-03,  9.40731520e-03,  3.44695519e-03,
        -9.99200722e-15, -1.40165657e-15],
       [ 1.92484083e-02,  2.93514622e-02,  1.33688237e-02,
         1.61293394e-02,  2.26379318e-02,  8.69892125e-03,
         6.43142333e-02,  8.33603108e-03,  8.61120104e-03,
         1.56738966e-02,  4.43509226e-02,  3.19753739e-03,
         1.14

# Análisis del modelo.

In [40]:
# A los datos iniciales les asignamos la nueva columna cluster en la cual
# se asigna el clúster en el cual fué clasificado el dato.
df["cluster"]=df_clusters.predict(interests_z).tolist()

#clúster a los que pertenecen los primeros cinco datos.
df[["cluster","gender","age","friends"]].head(5)

,cluster,gender,age,friends
0,0,M,18.982,7
1,1,F,18.801,0
2,0,M,18.335,69
3,3,F,18.875,0
4,2,NaN,18.995,10


In [42]:
# Agrupamos por clúster y calculamos la media de la edad.
df.groupby("cluster")['age'].mean()

cluster
0    17.432631
1    17.052040
2    17.269795
3    17.229694
4    17.072812
Name: age, dtype: float64

In [46]:
# Para cada clúster calculamos el número de mujeres.
# Clúster 1, 0 mujeres, clúster 2, 3095 mujeres, etc.
for a,b in df.groupby("cluster"):
    print(b['female'].sum())

0
3095
0
16472
2487


In [47]:
# Agrupamos por clúster y calculamos la media de amigos.
df.groupby("cluster")['friends'].mean()

cluster
0    24.926465
1    39.572536
2    26.139134
3    30.256982
4    33.431846
Name: friends, dtype: float64